In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [6]:
spark=SparkSession.builder\
        .appName('SparkApp')\
        .master('local[*]')\
        .getOrCreate()
spark.version

'4.0.1'

In [19]:
schema = StructType([
    StructField("player_name", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("day", StringType(), True),
    StructField("score_points", IntegerType(), True)
])

data = [
    ("Aron", "F", "2020-01-01", 17),
    ("Alice", "F", "2020-01-07", 23),
    ("Bajrang", "M", "2020-01-07", 7),
    ("Khali", "M", "2019-12-25", 11),
    ("Slaman", "M", "2019-12-30", 13),
    ("Joe", "M", "2019-12-31", 3),
    ("Jose", "M", "2019-12-18", 2),
    ("Priya", "F", "2019-12-31", 23),
    ("Priyanka", "F", "2019-12-30", 17)
]

scores_df=spark.createDataFrame(data,schema=schema)\
                .withColumn('day',to_date('day','yyyy-MM-dd'))

In [20]:
w=Window.partitionBy('gender').orderBy('gender','day')
scores_df=scores_df.withColumn('total',sum('score_points').over(w))
scores_df.show()

+-----------+------+----------+------------+-----+
|player_name|gender|       day|score_points|total|
+-----------+------+----------+------------+-----+
|   Priyanka|     F|2019-12-30|          17|   17|
|      Priya|     F|2019-12-31|          23|   40|
|       Aron|     F|2020-01-01|          17|   57|
|      Alice|     F|2020-01-07|          23|   80|
|       Jose|     M|2019-12-18|           2|    2|
|      Khali|     M|2019-12-25|          11|   13|
|     Slaman|     M|2019-12-30|          13|   26|
|        Joe|     M|2019-12-31|           3|   29|
|    Bajrang|     M|2020-01-07|           7|   36|
+-----------+------+----------+------------+-----+



In [22]:
scores_df[['gender','day','total']].show()

+------+----------+-----+
|gender|       day|total|
+------+----------+-----+
|     F|2019-12-30|   17|
|     F|2019-12-31|   40|
|     F|2020-01-01|   57|
|     F|2020-01-07|   80|
|     M|2019-12-18|    2|
|     M|2019-12-25|   13|
|     M|2019-12-30|   26|
|     M|2019-12-31|   29|
|     M|2020-01-07|   36|
+------+----------+-----+

